## IS between descriptors and embeddings

In [ ]:
%cd ..

In [ ]:

import os
import numpy as np
from utils.utils_notebook import get_MI_df, plot_cmap, LATEX_FIG_PATH, prerpocess_emb_name
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage


DATASET = "ZINC"
results_dir_list = ["run_4"]
COLUMS_SPLIT = "cond_modes"
METRIC = r"$\mathcal{I}_S$"

df = get_MI_df(DATASET, results_dir_list)

df.X = df.X.apply(prerpocess_emb_name)
df.Y = df.Y.apply(prerpocess_emb_name)


## Clustermap

In [ ]:
%matplotlib inline

df.loc[df.X == df.Y, "I(X->Y)/dim"] = df["I(X->Y)/dim"].max()

df_pivot = df.pivot_table(index="X", columns="Y", values="I(X->Y)/dim", aggfunc="mean")
link = linkage(df_pivot, method="median")


cg = sns.clustermap(
    df_pivot,
    row_linkage=link,
    col_linkage=link,
    cmap="coolwarm",
    figsize=(5.7, 7.2),
    robust=True,
    cbar_kws={"label": METRIC + r"$(Z\rightarrow U)$"},
    vmin=0.25,
    vmax = 1.3,
)

cg.ax_row_dendrogram.set_visible(False)
cg.ax_col_dendrogram.set_visible(False)
# move legend
cg.cax.set_visible(True)
#move legend
cg.cax.set_position([1, 0.3, 0.02, 0.45])

# Replace ylabel to Z and ylabel to X
cg.ax_heatmap.set_ylabel("Models Z")
cg.ax_heatmap.set_xlabel("Models U")
# Put the BertMTR models in bold red
for i, label in enumerate(cg.ax_heatmap.get_yticklabels()):
    if label.get_text().startswith("BertMTR") or label.get_text().startswith("MolR"):
        label.set_weight("bold")
        label.set_color("red")

#remove x ticks
cg.ax_heatmap.set_xticks([])

plt.savefig(
    f"{LATEX_FIG_PATH}/molecule/clustermap_MI.pdf",
    format="pdf",
    bbox_inches="tight",
)